<a href="https://colab.research.google.com/github/jcs-lambda/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install -U --quiet category_encoders pandas-profiling

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

# Assignment

In [5]:
df

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23035,4,OTHER,01 ONE FAMILY DWELLINGS,1,10965,276,NaN,A5,111-17 FRANCIS LEWIS BLVD,NaN,11429.0,1.0,0.0,1.0,"1,800",1224.0,1945.0,1,A5,510000,04/30/2019
23036,4,OTHER,09 COOPS - WALKUP APARTMENTS,2,169,29,NaN,C6,"45-14 43RD STREET, 3C",NaN,11104.0,0.0,0.0,0.0,0,0.0,1929.0,2,C6,355000,04/30/2019
23037,4,OTHER,10 COOPS - ELEVATOR APARTMENTS,2,131,4,NaN,D4,"50-05 43RD AVENUE, 3M",NaN,11377.0,0.0,0.0,0.0,0,0.0,1932.0,2,D4,375000,04/30/2019
23038,4,OTHER,02 TWO FAMILY DWELLINGS,1,8932,18,NaN,S2,91-10 JAMAICA AVE,NaN,11421.0,2.0,1.0,3.0,"2,078",2200.0,1931.0,1,S2,1100000,04/30/2019


In [6]:
df.describe()

,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,23040.000000,23040.000000,0.0,23039.000000,23039.000000,23039.000000,23039.000000,2.303900e+04,23005.000000,23040.000000,2.304000e+04
mean,4459.116493,353.948481,NaN,10791.186987,1.827423,0.286514,2.320891,3.508555e+03,1819.671724,1.624479,1.328133e+06
std,3713.269887,628.025255,NaN,1103.414210,10.090432,5.778224,12.070554,2.218610e+04,488.376571,0.803898,1.025394e+07
min,1.000000,1.000000,NaN,0.000000,0.000000,-148.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000e+00
25%,1341.000000,22.000000,NaN,10306.000000,0.000000,0.000000,1.000000,4.940000e+02,1920.000000,1.000000,0.000000e+00
50%,3546.000000,49.000000,NaN,11211.000000,1.000000,0.000000,1.000000,1.356000e+03,1940.000000,1.000000,4.550000e+05
75%,6673.750000,375.000000,NaN,11360.000000,2.000000,0.000000,2.000000,2.280000e+03,1965.000000,2.000000,8.750000e+05
max,16350.000000,9057.000000,NaN,11697.000000,750.000000,570.000000,755.000000,1.303935e+06,2019.000000,4.000000,8.500000e+08


In [7]:
df.describe(exclude='number')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,LAND_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,23040,23040,23040,23039,23039,23040,5201,22987,23040,23040
unique,5,11,44,10,146,22691,1724,3652,147,120
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,D4,100 JEROME STREET,4,0,D4,01/24/2019
freq,7494,19117,5061,11071,3408,6,99,7500,3408,480


In [8]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [9]:
df.isnull().sum()

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY               0
TAX_CLASS_AT_PRESENT                  1
BLOCK                                 0
LOT                                   0
EASE-MENT                         23040
BUILDING_CLASS_AT_PRESENT             1
ADDRESS                               0
APARTMENT_NUMBER                  17839
ZIP_CODE                              1
RESIDENTIAL_UNITS                     1
COMMERCIAL_UNITS                      1
TOTAL_UNITS                           1
LAND_SQUARE_FEET                     53
GROSS_SQUARE_FEET                     1
YEAR_BUILT                           35
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                            0
SALE_DATE                             0
dtype: int64

In [0]:
# LAND_DQUARE_FEET is strings, but should be an integer
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].fillna('0').str.replace('[^0-9]', '').str.replace('^$', '0').astype(int)

In [0]:
# fill in nulls in GROSS_SQUARE_FEET with 0
df['GROSS_SQUARE_FEET'] = df['GROSS_SQUARE_FEET'].fillna(0).astype(int)

In [0]:
# YEAR_BUILT is floats, but should be ints. fill nans with 0
df['YEAR_BUILT'] = df['YEAR_BUILT'].fillna(0).astype(int)

In [0]:
# ZIP_CODE is numeric, but should bbe categorical. fill in nans with 0; replace '0' with '00000'
df['ZIP_CODE'] = df['ZIP_CODE'].fillna(0).astype(int).astype(str).replace('^0$', '00000')

In [0]:
# TAX_CLASS_AT_TIME_OF_SALE is numeric, but should be categorical
df['TAX_CLASS_AT_TIME_OF_SALE'] = df['TAX_CLASS_AT_TIME_OF_SALE'].astype(str)

## Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.

In [15]:
condition_subset = (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') & (df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000)

df = df[condition_subset].copy()

# drop BUILDING_CLASS_CATEGORY because all observations contain the same value
df.drop(columns='BUILDING_CLASS_CATEGORY', inplace=True)

df

,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230,1.0,0.0,1.0,6800,1325,1930,1,A9,550000,01/01/2019
61,4,OTHER,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427,1.0,0.0,1.0,4000,2001,1940,1,A1,200000,01/01/2019
78,2,OTHER,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461,1.0,0.0,1.0,3500,2043,1925,1,A1,810000,01/02/2019
108,3,OTHER,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226,1.0,0.0,1.0,4000,2680,1899,1,A1,125000,01/02/2019
111,3,OTHER,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203,1.0,0.0,1.0,1710,1872,1940,1,A5,620000,01/02/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23029,4,OTHER,1,13215,3,NaN,A2,244-15 135 AVENUE,NaN,11422,1.0,0.0,1.0,3300,1478,1925,1,A2,635000,04/30/2019
23031,4,OTHER,1,11612,73,NaN,A1,10919 132ND STREET,NaN,11420,1.0,0.0,1.0,2400,1280,1930,1,A1,514000,04/30/2019
23032,4,OTHER,1,11808,50,NaN,A0,135-24 122ND STREET,NaN,11420,1.0,0.0,1.0,4000,1333,1945,1,A0,635000,04/30/2019
23033,4,OTHER,1,12295,23,NaN,A1,134-34 157TH STREET,NaN,11434,1.0,0.0,1.0,2500,1020,1935,1,A1,545000,04/30/2019


## Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [16]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True, errors='coerce')
print(f"null dates: {df['SALE_DATE'].isnull().sum()}")
print(f"min date: {df['SALE_DATE'].min()}")
print(f"max date: {df['SALE_DATE'].max()}")

null dates: 0
min date: 2019-01-01 00:00:00
max date: 2019-04-30 00:00:00


In [17]:
df_train = df[df['SALE_DATE'].dt.month < 4].copy()
df_test = df[df['SALE_DATE'].dt.month == 4].copy()
df_train.shape, df_test.shape

((2507, 20), (644, 20))

## Do one-hot encoding of categorical features.

In [18]:
df_train

,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230,1.0,0.0,1.0,6800,1325,1930,1,A9,550000,2019-01-01
61,4,OTHER,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427,1.0,0.0,1.0,4000,2001,1940,1,A1,200000,2019-01-01
78,2,OTHER,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461,1.0,0.0,1.0,3500,2043,1925,1,A1,810000,2019-01-02
108,3,OTHER,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226,1.0,0.0,1.0,4000,2680,1899,1,A1,125000,2019-01-02
111,3,OTHER,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203,1.0,0.0,1.0,1710,1872,1940,1,A5,620000,2019-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18129,5,OTHER,1,4081,44,NaN,A2,10 SEAFOAM STREET,NaN,10306,1.0,0.0,1.0,2400,921,1950,1,A2,330000,2019-03-29
18130,5,OTHER,1,2373,201,NaN,A5,74 MCVEIGH AVE,NaN,10314,1.0,0.0,1.0,2450,2128,1980,1,A5,690000,2019-03-29
18132,5,OTHER,1,1132,42,NaN,A1,479 VILLA AVENUE,NaN,10302,1.0,0.0,1.0,4361,1807,2018,1,A1,610949,2019-03-29
18134,5,OTHER,1,3395,37,NaN,A2,63 NUGENT AVENUE,NaN,10305,1.0,0.0,1.0,6000,621,1930,1,A2,520000,2019-03-29


In [19]:
df_train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
APARTMENT_NUMBER,1,1,RP.,1,NaT,NaT
TAX_CLASS_AT_TIME_OF_SALE,2507,1,1,2507,NaT,NaT
TAX_CLASS_AT_PRESENT,2507,2,1,2476,NaT,NaT
BOROUGH,2507,5,4,1204,NaT,NaT
NEIGHBORHOOD,2507,7,OTHER,2360,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,2507,11,A1,919,NaT,NaT
BUILDING_CLASS_AT_PRESENT,2507,13,A1,919,NaT,NaT
SALE_DATE,2507,68,2019-01-31 00:00:00,78,2019-01-01,2019-03-30
ZIP_CODE,2507,122,10312,115,NaT,NaT
ADDRESS,2507,2497,117-45 125TH STREET,2,NaT,NaT


In [20]:
df_train.select_dtypes(exclude='number').isnull().sum()

BOROUGH                              0
NEIGHBORHOOD                         0
TAX_CLASS_AT_PRESENT                 0
BUILDING_CLASS_AT_PRESENT            0
ADDRESS                              0
APARTMENT_NUMBER                  2506
ZIP_CODE                             0
TAX_CLASS_AT_TIME_OF_SALE            0
BUILDING_CLASS_AT_TIME_OF_SALE       0
SALE_DATE                            0
dtype: int64

In [21]:
for column in df_train.select_dtypes(exclude='number').columns:
  if df_train[column].nunique() > 50:
    continue
  print(f'{column}: {df_train[column].unique()}')

BOROUGH: ['3' '4' '2' '5' '1']
NEIGHBORHOOD: ['OTHER' 'FLUSHING-NORTH' 'EAST NEW YORK' 'BEDFORD STUYVESANT'
 'FOREST HILLS' 'BOROUGH PARK' 'ASTORIA']
TAX_CLASS_AT_PRESENT: ['1' '1D']
BUILDING_CLASS_AT_PRESENT: ['A9' 'A1' 'A5' 'A0' 'A2' 'A3' 'S1' 'A4' 'A6' 'A8' 'B2' 'S0' 'B3']
APARTMENT_NUMBER: [nan 'RP.']
TAX_CLASS_AT_TIME_OF_SALE: ['1']
BUILDING_CLASS_AT_TIME_OF_SALE: ['A9' 'A1' 'A5' 'A0' 'A2' 'A3' 'S1' 'A4' 'A6' 'A8' 'S0']


In [22]:
data = {
    'nulls':df_train.select_dtypes(include='number').isnull().sum(),
    'unique':df_train.select_dtypes(include='number').nunique(),
    'count':df_train.select_dtypes(include='number').notnull().sum()
}

pd.DataFrame(data)

,nulls,unique,count
BLOCK,0,2060,2507
LOT,0,304,2507
EASE-MENT,2507,0,0
RESIDENTIAL_UNITS,0,2,2507
COMMERCIAL_UNITS,0,3,2507
TOTAL_UNITS,0,4,2507
LAND_SQUARE_FEET,0,887,2507
GROSS_SQUARE_FEET,0,922,2507
YEAR_BUILT,0,86,2507
SALE_PRICE,0,880,2507


In [0]:
target = 'SALE_PRICE'

features = [
            'BOROUGH', 'NEIGHBORHOOD', 'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_TIME_OF_SALE',
            'BUILDING_CLASS_AT_PRESENT', 'ZIP_CODE', 'BLOCK', 'LOT', 'RESIDENTIAL_UNITS',
            'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT'
]

X_train = df_train[features]
X_test = df_test[features]

y_train = df_train[target]
y_test = df_test[target]

In [24]:
from category_encoders import OneHotEncoder

encoder = OneHotEncoder(use_cat_names=True)

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

X_train_encoded.shape, X_test_encoded.shape

((2507, 168), (644, 168))

## Do feature selection with SelectKBest.
## Fit a ridge regression model with multiple features. 

Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)

## Get mean absolute error for the test set.

In [25]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train_encoded.columns)+1):
  selector = SelectKBest(score_func=f_regression, k=k)
  X_train_selected = selector.fit_transform(X_train_encoded, y_train)
  X_test_selected = selector.transform(X_test_encoded)

  model = Ridge(normalize=True)
  model.fit(X_train_selected, y_train)
  y_pred = model.predict(X_test_selected)
  mae = mean_absolute_error(y_test, y_pred)

  if k == 1 or mae < best_mae:
    best_k = k
    best_mae = mae
    best_selector = selector
    best_model = model
  
print(f'Best (lowest) mean absolute error: ${best_mae:.2f}')
print(f'using {best_k} of {len(X_train_encoded.columns)} features:')
X_train_encoded.columns[best_selector.get_support()]

Best (lowest) mean absolute error: $143725.15
using 166 of 168 features:


Index(['BOROUGH_3', 'BOROUGH_4', 'BOROUGH_2', 'BOROUGH_5', 'BOROUGH_1',
       'NEIGHBORHOOD_OTHER', 'NEIGHBORHOOD_FLUSHING-NORTH',
       'NEIGHBORHOOD_EAST NEW YORK', 'NEIGHBORHOOD_FOREST HILLS',
       'NEIGHBORHOOD_BOROUGH PARK',
       ...
       'ZIP_CODE_11102', 'ZIP_CODE_10459', 'BLOCK', 'LOT', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT'],
      dtype='object', length=166)

# Stretch goals

##Instead of Ridge, try LinearRegression.

 Depending on how many features you select, your errors will probably blow up! 💥


In [26]:
from sklearn.linear_model import LinearRegression

for k in range(1, len(X_train_encoded.columns)+1):
  selector = SelectKBest(score_func=f_regression, k=k)
  X_train_selected = selector.fit_transform(X_train_encoded, y_train)
  X_test_selected = selector.transform(X_test_encoded)

  model = LinearRegression(n_jobs=-1)
  model.fit(X_train_selected, y_train)
  y_pred = model.predict(X_test_selected)
  mae = mean_absolute_error(y_test, y_pred)

  if k == 1 or mae < best_mae:
    best_k = k
    best_mae = mae
    best_selector = selector
    best_model = model
  
print(f'Best (lowest) mean absolute error: ${best_mae:.2f}')
print(f'using {best_k} of {len(X_train_encoded.columns)} features:')
X_train_encoded.columns[best_selector.get_support()]

Best (lowest) mean absolute error: $137973.74
using 96 of 168 features:


Index(['BOROUGH_3', 'BOROUGH_4', 'BOROUGH_2', 'BOROUGH_5',
       'NEIGHBORHOOD_OTHER', 'NEIGHBORHOOD_FLUSHING-NORTH',
       'NEIGHBORHOOD_EAST NEW YORK', 'NEIGHBORHOOD_FOREST HILLS',
       'NEIGHBORHOOD_BOROUGH PARK', 'NEIGHBORHOOD_ASTORIA',
       'TAX_CLASS_AT_PRESENT_1', 'TAX_CLASS_AT_PRESENT_1D',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A1',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A5',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A3',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S1',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A4',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A6',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A8',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S0', 'BUILDING_CLASS_AT_PRESENT_A1',
       'BUILDING_CLASS_AT_PRESENT_A5', 'BUILDING_CLASS_AT_PRESENT_A3',
       'BUILDING_CLASS_AT_PRESENT_S1', 'BUILDING_CLASS_AT_PRESENT_A4',
       'BUILDING_CLASS_AT_PRESENT_A6', 'BUILDING_CLASS_AT_PRESENT_A8',
       'BUILDING_CLASS_AT_PRESENT_S0', 'ZIP_CODE_11230', 'ZIP_CODE_11229',
       'ZIP_CODE_11364'

## Instead of Ridge, try RidgeCV

In [27]:
from sklearn.linear_model import RidgeCV

alphas = [0.01, 0.1, 1.0, 10.0, 100.0]
alphas = [i/10 for i in range(1, 11)]
alphas = [i/100 for i in range(1, 11)]
alphas = [i/100+.1 for i in range(1, 11)]
print(alphas)

for k in range(1, len(X_train_encoded.columns)+1):
  selector = SelectKBest(score_func=f_regression, k=k)
  X_train_selected = selector.fit_transform(X_train_encoded, y_train)
  X_test_selected = selector.transform(X_test_encoded)

  model = RidgeCV(alphas=alphas, normalize=True)
  model.fit(X_train_selected, y_train)
  y_pred = model.predict(X_test_selected)
  mae = mean_absolute_error(y_test, y_pred)

  if k == 1 or mae < best_mae:
    best_k = k
    best_mae = mae
    best_selector = selector
    best_model = model
  
print(f'Best (lowest) mean absolute error: ${best_mae:.2f}')
print(f'Best alpha from RidgeCV: {best_model.alpha_}')
print(f'using {best_k} of {len(X_train_encoded.columns)} features:')
X_train_encoded.columns[best_selector.get_support()]

[0.11, 0.12000000000000001, 0.13, 0.14, 0.15000000000000002, 0.16, 0.17, 0.18, 0.19, 0.2]
Best (lowest) mean absolute error: $136487.63
Best alpha from RidgeCV: 0.11
using 96 of 168 features:


Index(['BOROUGH_3', 'BOROUGH_4', 'BOROUGH_2', 'BOROUGH_5',
       'NEIGHBORHOOD_OTHER', 'NEIGHBORHOOD_FLUSHING-NORTH',
       'NEIGHBORHOOD_EAST NEW YORK', 'NEIGHBORHOOD_FOREST HILLS',
       'NEIGHBORHOOD_BOROUGH PARK', 'NEIGHBORHOOD_ASTORIA',
       'TAX_CLASS_AT_PRESENT_1', 'TAX_CLASS_AT_PRESENT_1D',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A1',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A5',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A3',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S1',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A4',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A6',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A8',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S0', 'BUILDING_CLASS_AT_PRESENT_A1',
       'BUILDING_CLASS_AT_PRESENT_A5', 'BUILDING_CLASS_AT_PRESENT_A3',
       'BUILDING_CLASS_AT_PRESENT_S1', 'BUILDING_CLASS_AT_PRESENT_A4',
       'BUILDING_CLASS_AT_PRESENT_A6', 'BUILDING_CLASS_AT_PRESENT_A8',
       'BUILDING_CLASS_AT_PRESENT_S0', 'ZIP_CODE_11230', 'ZIP_CODE_11229',
       'ZIP_CODE_11364'

## try scikit-learn pipelines

In [31]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import mutual_info_regression

pipeline = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    SelectKBest(score_func=mutual_info_regression, k=96),
    Ridge(alpha=0.11, normalize=True)
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

print(f'test mae: ${mae:.2f}')

test mae: $139758.15


In [36]:
for score_func in [f_regression, mutual_info_regression]:
  for k in [10, 20, 40, 60, 80, 100, 120]:
    pipeline = make_pipeline(
        OneHotEncoder(use_cat_names=True),
        SelectKBest(score_func=score_func, k=k),
        Ridge(alpha=0.11, normalize=True)
    )
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'{score_func.__name__}, k={k}, mae: ${mae:.2f}')

f_regression, k=10, mae: $170282.63
f_regression, k=20, mae: $152831.80
f_regression, k=40, mae: $143831.91
f_regression, k=60, mae: $141096.47
f_regression, k=80, mae: $137851.26
f_regression, k=100, mae: $137023.83
f_regression, k=120, mae: $136635.89
mutual_info_regression, k=10, mae: $172330.05
mutual_info_regression, k=20, mae: $157339.92
mutual_info_regression, k=40, mae: $154508.27
mutual_info_regression, k=60, mae: $143314.81
mutual_info_regression, k=80, mae: $143491.77
mutual_info_regression, k=100, mae: $143492.34
mutual_info_regression, k=120, mae: $144809.17


In [37]:
for score_func in [f_regression, mutual_info_regression]:
  for k in [10, 20, 40, 60, 80, 100, 120, 140, 150]:
    pipeline = make_pipeline(
        OneHotEncoder(use_cat_names=True),
        SelectKBest(score_func=score_func, k=k),
        LinearRegression(n_jobs=-1)
    )
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'{score_func.__name__}, k={k}, mae: ${mae:.2f}')

f_regression, k=10, mae: $170236.83
f_regression, k=20, mae: $152918.09
f_regression, k=40, mae: $143366.61
f_regression, k=60, mae: $141112.21
f_regression, k=80, mae: $138420.72
f_regression, k=100, mae: $138813.34
f_regression, k=120, mae: $139108.61
f_regression, k=140, mae: $1078085198.01
f_regression, k=150, mae: $41031371633.00
mutual_info_regression, k=10, mae: $166562.43
mutual_info_regression, k=20, mae: $150356.27
mutual_info_regression, k=40, mae: $148095.49
mutual_info_regression, k=60, mae: $149749.57
mutual_info_regression, k=80, mae: $145267.14
mutual_info_regression, k=100, mae: $145571.27
mutual_info_regression, k=120, mae: $111755437869.25
mutual_info_regression, k=140, mae: $50034326194.85
mutual_info_regression, k=150, mae: $15662842829.02
